In [1]:
import time
import getpass
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [9]:
class InstaBot:
    def __init__(self):
        self.driver = driver
        self.current_state = 'login_screen'
    
    def get_current_state(self):
        return self.current_state

    def login(self, uname, pword):
        if self.current_state == 'login_screen':
            username = driver.find_element(By.NAME, 'username')
            password = driver.find_element(By.NAME, 'password')
            username.clear()
            password.clear()
            submit = driver.find_element(By.XPATH, '//button[@class="sqdOP  L3NKy   y3zKF     "]')
            password.send_keys(pword)
            username.send_keys(uname)
            submit.click()
            self.uname = uname
            try:
                err = driver.find_element(By.XPATH, '//p[@id="slfErrorAlert"]')
                print("Could not login, error was raised, please check login details and try again later")
                return "quit"
            except NoSuchElementException:
                self.current_state = 'logged_in:'+uname
                return "init"
        else:
            print('Login unsuccessful')
    
    def logout(self):
        if self.current_state.split(':')[0] != 'browsing_followers':
            drop_down = driver.find_element(By.XPATH, '//img[@alt="'+self.uname+'\'s profile picture"]')
            drop_down.click()
            logout_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[contains(text(),"Log Out")]')))
            logout_btn.click()
            self.current_state = 'login_screen'
        else:
            print("Cannot logout, try closing manually")
    
    def search_handles(self, query):
        search = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')
        search.clear()
        search.send_keys(" ")
        search.clear()
        search.send_keys(query) 
        handle = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@role="none"]')))
        handles = driver.find_elements(By.XPATH, '//div[@role="none"]')
        for handle in handles:
            if handle.text.split()[0][0]!="#" :
                print(handle.text.split()[0])
        
        self.current_state = 'searching_handles:'+query
    
    def go_to_profile(self, profile):
        search = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')
        search.clear()
        search.send_keys(" ")
        time.sleep(2)
        search.send_keys(profile)
        time.sleep(3)
        handle = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@role="none"]')))
        handle.click()
        self.current_state = 'browsing_handle:'+profile
    
    def follow(self, profile):
        if self.current_state == "browsing_handle:"+profile:
            try:
                follow_btn = driver.find_element(By.XPATH, '//button[@class="_acan _acap _acas"]')
                follow_btn.click()
            except:
                print("Already following account. Did you mean 'unfollow(profile)'?'")
        else:
            self.go_to_profile(profile)
            self.follow(profile)
    
    def unfollow(self, profile):
        if self.current_state == "browsing_handle:"+profile:
            try:
                follow = driver.find_element(By.XPATH, '//button[@class="_acan _acap _acas"]')
                print("Not following this account. Did you mean 'follow(profile)'?")
            except:
                print("-")
            try:
                alert = driver.find_element(By.XPATH, '//div[@class="_ab8w  _ab94 _ab99 _ab9h _ab9m _ab9p  _ab9- _abcm"]')
                alert.click()
                unfollow = driver.find_element(By.XPATH, '//button[@class="_a9-- _a9-_"]')
                unfollow.click()
            except:
                print("Error")
        else:
            self.go_to_profile(profile)
            self.unfollow(profile)
    
    def find_followers(self, profile, number_of_followers):
        time.sleep(3)
        if self.current_state != 'browsing_handle:'+profile:
            self.go_to_profile(profile)
            time.sleep(5)
            self.find_followers(profile, number_of_followers)
        else:
            if self.current_state != 'browsing_followers:'+profile:
                follower = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,  '//a[@class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl _a6hd"]')))
                follower.click()
                current_state = 'browsing_followers:'+profile
            while True:
                followers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//span[@class="_aacl _aaco _aacw _aacx _aad7 _aade"]')))
                if len(followers) > number_of_followers:
                    break
                else:
                    scroll = driver.find_element(By.XPATH, '//div[@class="_aano"]')
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll)
                    time.sleep(1)
            return_followers = [i.text for i in followers]  
            for i in return_followers:
                print(i, end="||")
            driver.back()
            self.current_status = 'browsing_handle:'+profile
    
    def view_story(self):
        try:
            story = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//div[@class="_aarf _aarg"]')))
            seen = driver.find_element(By.XPATH, '//canvas[@class="_aarh"]')
            if int(seen.get_attribute('height')) == 210:
                print("Showing story")
                story.click()
            else:
                flag = input("Already viewed story, do you want to view the story again?(Yes/No): ")
                if flag.lower() == "yes":
                    story.click()
        except NoSuchElementException:
            try:
                story = driver.find_element(By.XPATH, '//div[@class="_aarf"]')
                print("No story at profile ", b1.get_current_status().split(':')[-1])
            except:
                pass
    
    def like_posts(self, n):
        time.sleep(3)
        post = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="_aagw"]')))
        post.click()
        next_posts = driver.find_elements(By.XPATH, '//div[contains(@class,"_aaqg _aaqh")]//button[contains(@type,"button")]')
        ctr = n
        while True:
            try:
                time.sleep(3)
                ctr -= 1
                if ctr < 0:
                    raise NoSuchElementException
                liked = driver.find_elements(By.XPATH, '//span[contains(@class,"_aamw")]//button[contains(@type,"button")]/div')
                if len(liked) == 1:
                    print("Post ", ctr, " is already liked")
                else:
                    print("Post", ctr, "liked")
                    btn = driver.find_element(By.XPATH, '//span[@class="_aamw"]//button[@type="button"]')
                    btn.click()
                next_post = driver.find_element(By.XPATH, '//div[contains(@class,"_aaqg _aaqh")]//button[contains(@type,"button")]')
                next_post.click()
            except NoSuchElementException:
                try:
                    close = driver.find_element(By.XPATH, '//div[@class="oajrlxb2 rq0escxv p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x i1ao9s8h esuyzwwr f1sip0of lzcic4wl li38xygf futnfnd5 c9k30104 n3a6jdvc d7m63fex f8tn558r kkot5xxs bpozc334 q0p5rdf8 nv6ama0i c2oockdd tcrfgq0d nttjwlih"]')
                    close.click()
                except:
                    pass
                break
                
    def unlike_posts(self, n):
        post = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="_aagw"]')))
        post.click()
        next_posts = driver.find_elements(By.XPATH, '//div[contains(@class,"_aaqg _aaqh")]//button[contains(@type,"button")]')
        ctr = n
        while True:
            try:
                time.sleep(2)
                ctr -= 1
                if ctr < 0:
                    raise NoSuchElementException
                liked = driver.find_elements(By.XPATH, '//span[contains(@class,"_aamw")]//button[contains(@type,"button")]/div')
                if len(liked) == 1:
                    print("Post ", ctr, " unliked")
                    btn = driver.find_element(By.XPATH, '//span[@class="_aamw"]//button[@type="button"]')
                    btn.click()
                else:
                    print("Post", ctr, "is not liked")
                    liked[0].click()
                next_post = driver.find_element(By.XPATH, '//div[contains(@class,"_aaqg _aaqh")]//button[contains(@type,"button")]')
                next_post.click()
            except NoSuchElementException:
                try:
                    close = driver.find_element(By.XPATH, '//div[@class="oajrlxb2 rq0escxv p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x i1ao9s8h esuyzwwr f1sip0of lzcic4wl li38xygf futnfnd5 c9k30104 n3a6jdvc d7m63fex f8tn558r kkot5xxs bpozc334 q0p5rdf8 nv6ama0i c2oockdd tcrfgq0d nttjwlih"]')
                    close.click()
                except:
                    pass
                break

# Project as per requirements

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://instagram.com')
current_status = "init"
b = InstaBot()
# uname = input('Enter Instagram Username: ')
# pword = getpass.getpass()
uname = SAMPLE_USERNAME
pword = SAMPLE_PASSWORD
# 1. Login to your Instagram Handle
# Submit with sample username and password
b.login(uname, pword)
time.sleep(5)
# 2. Type for “food” in search bar and print all the names of the Instagram Handles that are displayed in list after typing “food”
# Note : Make sure to avoid printing hashtags
b.search_handles("food")
time.sleep(15)
# 3. Open profile of “So Delhi” 
# Follow/Unfollow given handle - 
# Open the Instagram Handle of “So Delhi”
# Start following it. Print a message if you are already following
# After following, unfollow the instagram handle. Print a message if you have already unfollowed.
b.go_to_profile("So Delhi")
time.sleep(10)
b.follow("So Delhi")
time.sleep(10)
b.unfollow("So Delhi")
time.sleep(20)
# 4. Like/Unlike posts
# Liking the top 30 posts of the ‘dilsefoodie'. Print message if you have already liked it.
# Unliking the top 30 posts of the ‘dilsefoodie’. Print message if you have already unliked it.
b.go_to_profile("dilsefoodie")
time.sleep(20)
b.like_posts(30)
time.sleep(30)
driver.refresh()
b.unlike_posts(30)
time.sleep(30)
# 5. Extract list of followers
# Extract the usernames of the first 500 followers of ‘foodtalkindia’ and ‘sodelhi’.
# Now print all the followers of “foodtalkindia” that you are following but those who don’t follow you.
b.go_to_profile("foodtalkindia")
driver.refresh()
time.sleep(5)
print("Followers of Food Talk India: ")
b.find_followers("foodtalkindia", 500)
time.sleep(10)
print("\nFollowers of So Delhi: ")
b.go_to_profile("sodelhi")
driver.refresh()
time.sleep(5)
b.find_followers("sodelhi", 500)
time.sleep(10)
# 6. Check the story of ‘coding.ninjas’. Consider the following Scenarios and print error messages accordingly -
# If You have already seen the story.
# Or The user has no story.
# Or View the story if not yet seen.
b.go_to_profile("coding ninjas")
time.sleep(5)
b.view_story()

# Menu Driven version of project

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://instagram.com')
current_status = "init"
b1 = InstaBot()
uname = input('Enter Instagram Username: ')
pword = getpass.getpass()
choice = b1.login(uname, pword)
while(choice != "quit"):
    print("CURRENT STATE: ", b1.get_current_state())
    choice = input("1. Search handles related to query\n2. Search particular profile\n3. Follow given profile \n4. Unfollow given profile \n5. Extract followers of a given profile\n6. View story of profile\n7. Like first n posts of a profile\n8. Unlike first n posts of a profile\n9. Logout and resume\n10. Logout and quit\nENTER: ")
    
    if choice.strip() == '1':
        query = input("Enter query to be searched: ")
        b1.search_handles(query)
        
    elif choice.strip() == '2':
        profile = input("Enter profile to be searched: ")
        b1.go_to_profile(profile)
    
    elif choice.strip() == '3':
        profile = input("Enter profile to be followed: ")
        b1.follow(profile)
    
    elif choice.strip() == '4':
        profile = input("Enter profile to be unfollowed: ")
        b1.unfollow(profile)
    
    elif choice.strip() == '5':
        profile = input("Enter profile whose followers are to be extracted: ")
        n = int(input("Enter number of followers to be extracted: "))
        b1.find_followers(profile, n)
    
    elif choice.strip() == '6':
        b1.view_story()
    
    elif choice.strip() == '7':
        n = int(input("Enter posts to be liked: "))
        b1.like_posts(n)
        
    elif choice.strip() == '8':
        n = int(input("Enter posts to be unliked: "))
        b1.unlike_posts(n)
        
    elif choice.strip() == '9':
        b1.logout()
        uname = input('Enter Instagram Username: ')
        pword = getpass.getpass()
        b1.login(uname, pword)
        
    elif choice.strip() == '10':
        b1.logout()
        choice = 'quit'
        driver.quit()
    
    else:
        print("Invalid input, please enter a valid input!")
    

C:\Users\Vyom\AppData\Local\Temp/ipykernel_17484/4196430510.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Enter Instagram Username: yemigone
········
CURRENT STATE:  logged_in:yemigone


In [ ]:


# Searching and Opening a profile using 


C:\Users\Vyom\AppData\Local\Temp/ipykernel_7976/1706405815.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Enter Instagram Username: yemigone
········
dilsefoodie
foodtalkindia
amritsar_foodblogger
foodzeee
delhifoodguide
food_lunatic
food
kashmirfoodgram
so_food_delhi
foodiesdelhite
punjabi_food_factory
foodie_incarnate
foodieveggie
foodofvadodara
delhifoodwalks
foodiesfood_court
food_era_
foodiewe
foodmapjaipur
yourfoodlab
foodiesince96
foodofgoa
swadisht.food
delhieater
foodiesafarii
delhifoodgrammer
shreo_s_food_diary
ludhiana_foodiess
the.food.busters
kuljyoti.dhingra
hyderabad.food.diaries
food_travel_etc
foodsanyasi
delhifoodblogger
foodiliciousmoments
foodsandflavorsbyshilpi
foodiee_says
thaparfoodfestival
kolkatafoodie
foodcomics_
thegreatindianfoodie
thewickedsoul
foodelhi
foodieinboots
foodtalesofficial
amritsarfood
food_realm_
pb11foodies
epicur3a
amritsar.food.diaries
foodiebyheart2.0
food_junc
-
Post 29 liked
Post 28 liked
Post 27 liked
Post 26 liked
Post 25 liked
Post 24 liked
Post 23 liked
Post 22 liked
Post 21 liked
Post 20 liked
Post 19 liked
Post 18 liked
Post 17 liked
Po

groupofkataria||noddy_sahill||the_caramelcrunch||rajnisharma_17_||rahmankaka_07||classid22||i_akshit||jalmehta||a_hot_cup_of_coffee||ysanskritiii||_sassy_worldthings||magic__soul__16||rahulrander||ranulfsessions||rsawhns||__m.e.m.e.w.o.r.l.d__||ayeza_fatima011||naveenkriplani||ashu_ajay_143||sarika_v111||focusstudios.in||aashish.6190||thepixelvert||bismillah_accissories||vinodyadhu15||lazyfat2.0||akram.shikoh||harshitasharma021||vatsal_1206||b1n1y4m||chintanmalhotra||syeds8121||praneeth_peruvala||5tbssd||parthgupta7121||mubydam||viratinteriors||siddhi.gala_||vxrsshh11||saesha_bakes||simranr605||theredfrog.hkv||varma.idk||_rahul_sinha||sydrorng||mohammedalimir||sahilxdesign||ksoumadeep||humanofdecor||vidya.raishetty||himu.k05||abhisnehathakur||homeincharge||ashishthukral5||b.sandekar||muskaanmehra1||ikendra_preet||sam112358||sigmarambler6||channi_33433||aaruuuuuuuuuuu__001||theojaslambhe||sanjyokirasoiclass||thegiftingsolutions||shubham.sananda||_nihhhhkmnn||mandeepkaursaiba9||radhe_ver